## License
This file is part of the project megFingerprinting. All of megFingerprinting code is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version. megFingerprinting is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details. You should have received a copy of the GNU General Public License along with megFingerprinting. If not, see https://www.gnu.org/licenses/.

In [3]:
import difflib
from fuzzywuzzy import fuzz
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
from os import listdir
from os.path import isfile, join
import pandas as pd
import re
import seaborn as sns
import scipy as sp
import scipy.io as sio
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from scipy import stats
from scipy.stats import pearsonr
sns.set(font_scale=2)
sns.set_style("whitegrid")
sns.set_palette(sns.color_palette("husl", 8))
import re

# I. Subject Identifiability: Original Feature Space vs PCA Reconstructed

In [6]:
# Parameters
n_subs = 133 # Change here to get number of participants! 
n_feats = int(68*115)
n_measurements = 2

In [7]:
# Warangle data set into two big feature matrices
def prune_subject_csv(filename):
    '''
    This function takes in the subject's csv file from MATLAB, takes out the 
    doubled correlations (because of symmetry) and outputs a numpy array ready to be concatenated
    in the grand feature matrix
    Args:
        filename (string): Name of the csv matrix
    Returns: 
        sub_feat (np.array): Subject's features 
    '''
    #print(filename)
    #print(filename[73:78])
    sub_feat = np.zeros([1, (n_feats)+1]) # Number of unique values in corr matrix + subject label
    psd_matrix = pd.read_csv(filename, header=None)
    mat=np.asmatrix(psd_matrix)
    sub_feat[0, :-1]=mat[:,0:451].flatten()
    sub_feat[0, -1] = int(filename[71:76])    
    return sub_feat


dataset1= [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3,3,3,3,3,3,3, 4, 4, 4,4,4,4,4,4,4, 5,5,5,5,5,5,5,5, 6,6,6,6,6,6,6,7,7,7,7,7,7,8,8,8,8,8,9,9,9,9,10,10,10, 11,11, 12]
dataset2= [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 4, 5, 6, 7,8, 9, 10, 11, 12, 13,  5, 6, 7,8, 9, 10, 11, 12, 13, 6,7, 8, 9, 10, 11, 12, 13, 7, 8, 9, 10, 11, 12, 13, 8, 9, 10, 11, 12, 13, 9,10 ,11,12,13,10,11,12,13, 11,12,13, 12, 13,13 ]

target_from_database = np.zeros([78, 2])
self_corr = np.zeros([133, 78])
cross_corr = np.zeros([133, 78])
for j in range(0,78):

    # Get n subjects: both training and testing datasets
    onlyfiles = [f for f in listdir('/Users/jason/Documents/PD_fingerprinting/2023_data/short_aperiodic/') if isfile(join('/Users/jason/Documents/PD_fingerprinting/2023_data/short_aperiodic/', f))]
    sub_target = np.zeros([n_subs, (n_feats)+1])
    sub_database = np.zeros([n_subs, (n_feats)+1])
    iv = 0
    it = 0
    for iFile in sorted(onlyfiles)[0:(n_subs*13)]: 
        sub = '/Users/jason/Documents/PD_fingerprinting/2023_data/short_aperiodic/' + iFile
        #print(sub[97:100])
        #print(re.sub("[^0-9]", "", sub[97:100]))
        if re.sub("[^0-9]", "", sub[95:98]) == str(dataset1[j]):
            sub_database[it, :] = prune_subject_csv(sub)
            it += 1
        if re.sub("[^0-9]", "", sub[95:98]) == str(dataset2[j]):
            sub_target[iv, :] = prune_subject_csv(sub)
            iv += 1


    # Correlations can be computed as the dot product between two z-scored vectors
    z_target = sp.stats.zscore(sub_target[:, :-1], axis = 1)
    z_database = sp.stats.zscore(sub_database[:,:-1], axis = 1)
    predictions = z_target.dot(z_database.transpose()) / (sub_database.shape[1] - 1) # target, database
    target_from_database[j,0] = accuracy_score(range(n_subs), predictions.argmax(axis = 1))
    target_from_database[j,1] = accuracy_score(range(n_subs), predictions.argmax(axis = 0))
    self_corr[0:133,j] = predictions.diagonal()
    np.fill_diagonal(predictions, 0)
    cross_corr[0:133,j] = predictions.mean(axis=1 )*(133/132)

    print('When predicting the target from the database, we get a ' + str(target_from_database[j,0]*100)[0:5] + '% accuracy')
    print('When predicting the database from the target, we get a ' + str(target_from_database[j,1]*100)[0:5] + '% accuracy')


When predicting the target from the database, we get a 96.99% accuracy
When predicting the database from the target, we get a 95.48% accuracy
When predicting the target from the database, we get a 89.47% accuracy
When predicting the database from the target, we get a 86.46% accuracy
When predicting the target from the database, we get a 88.72% accuracy
When predicting the database from the target, we get a 90.22% accuracy
When predicting the target from the database, we get a 84.96% accuracy
When predicting the database from the target, we get a 84.96% accuracy
When predicting the target from the database, we get a 84.21% accuracy
When predicting the database from the target, we get a 80.45% accuracy
When predicting the target from the database, we get a 81.95% accuracy
When predicting the database from the target, we get a 81.20% accuracy
When predicting the target from the database, we get a 81.95% accuracy
When predicting the database from the target, we get a 80.45% accuracy
When p

When predicting the target from the database, we get a 75.93% accuracy
When predicting the database from the target, we get a 75.93% accuracy
When predicting the target from the database, we get a 57.14% accuracy
When predicting the database from the target, we get a 58.64% accuracy
When predicting the target from the database, we get a 54.13% accuracy
When predicting the database from the target, we get a 57.14% accuracy
When predicting the target from the database, we get a 53.38% accuracy
When predicting the database from the target, we get a 53.38% accuracy
When predicting the target from the database, we get a 55.63% accuracy
When predicting the database from the target, we get a 57.89% accuracy
When predicting the target from the database, we get a 83.45% accuracy
When predicting the database from the target, we get a 80.45% accuracy
When predicting the target from the database, we get a 60.90% accuracy
When predicting the database from the target, we get a 61.65% accuracy
When p

In [8]:
dppca = pd.DataFrame(target_from_database)
dppca['dataset1']=dataset1
dppca['dataset2']=dataset2
dppca.to_csv("2023_aperiodic_complete_subj_short_recordings_NEW.csv", header=False, index=False)

self_corr_pd = pd.DataFrame(self_corr)
self_corr_pd.to_csv("2023_aperiodic_short_recordings_NEW_self_corr.csv", header=False, index=False)

cross_corr_pd = pd.DataFrame(cross_corr)
cross_corr_pd.to_csv("2023_aperiodic_short_recordings_NEW_cross_corr.csv", header=False, index=False)





In [10]:
# Parameters
n_subs = 54 # Change here to get number of participants! 
n_feats = int(68*115)
n_measurements = 2

# Warangle data set into two big feature matrices
def prune_subject_csv(filename):
    '''
    This function takes in the subject's csv file from MATLAB, takes out the 
    doubled correlations (because of symmetry) and outputs a numpy array ready to be concatenated
    in the grand feature matrix
    Args:
        filename (string): Name of the csv matrix
    Returns: 
        sub_feat (np.array): Subject's features 
    '''
    #print(filename)
    #print(filename[73:78])
    sub_feat = np.zeros([1, (n_feats)+1]) # Number of unique values in corr matrix + subject label
    psd_matrix = pd.read_csv(filename, header=None)
    mat=np.asmatrix(psd_matrix)
    sub_feat[0, :-1]=mat[:,0:451].flatten()
    sub_feat[0, -1] = int(filename[71:76])    
    return sub_feat


dataset1= [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3,3,3,3,3,3,3, 4, 4, 4,4,4,4,4,4,4, 5,5,5,5,5,5,5,5, 6,6,6,6,6,6,6,7,7,7,7,7,7,8,8,8,8,8,9,9,9,9,10,10,10, 11,11, 12]
dataset2= [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 4, 5, 6, 7,8, 9, 10, 11, 12, 13,  5, 6, 7,8, 9, 10, 11, 12, 13, 6,7, 8, 9, 10, 11, 12, 13, 7, 8, 9, 10, 11, 12, 13, 8, 9, 10, 11, 12, 13, 9,10 ,11,12,13,10,11,12,13, 11,12,13, 12, 13,13 ]

target_from_database = np.zeros([78, 2])
self_corr = np.zeros([54, 78])
cross_corr = np.zeros([54, 78])
for j in range(0,78):

    # Get n subjects: both training and testing datasets
    onlyfiles = [f for f in listdir('/Users/jason/Documents/PD_fingerprinting/2023_data/short_aperiodic/') if isfile(join('/Users/jason/Documents/PD_fingerprinting/2023_data/short_aperiodic/', f))]
    sub_target = np.zeros([n_subs, (n_feats)+1])
    sub_database = np.zeros([n_subs, (n_feats)+1])
    iv = 0
    it = 0
    for iFile in sorted(onlyfiles)[0:(n_subs*13)]: 
        sub = '/Users/jason/Documents/PD_fingerprinting/2023_data/short_aperiodic/' + iFile
        #print(sub[97:100])
        #print(re.sub("[^0-9]", "", sub[97:100]))
        if re.sub("[^0-9]", "", sub[95:98]) == str(dataset1[j]):
            sub_database[it, :] = prune_subject_csv(sub)
            it += 1
        if re.sub("[^0-9]", "", sub[95:98]) == str(dataset2[j]):
            sub_target[iv, :] = prune_subject_csv(sub)
            iv += 1


    # Correlations can be computed as the dot product between two z-scored vectors
    z_target = sp.stats.zscore(sub_target[:, :-1], axis = 1)
    z_database = sp.stats.zscore(sub_database[:,:-1], axis = 1)
    predictions = z_target.dot(z_database.transpose()) / (sub_database.shape[1] - 1) # target, database
    target_from_database[j,0] = accuracy_score(range(n_subs), predictions.argmax(axis = 1))
    target_from_database[j,1] = accuracy_score(range(n_subs), predictions.argmax(axis = 0))
    self_corr[0:133,j] = predictions.diagonal()
    np.fill_diagonal(predictions, 0)
    cross_corr[0:133,j] = predictions.mean(axis=1 )*(54/53)

    print('When predicting the target from the database, we get a ' + str(target_from_database[j,0]*100)[0:5] + '% accuracy')
    print('When predicting the database from the target, we get a ' + str(target_from_database[j,1]*100)[0:5] + '% accuracy')


When predicting the target from the database, we get a 98.14% accuracy
When predicting the database from the target, we get a 98.14% accuracy
When predicting the target from the database, we get a 94.44% accuracy
When predicting the database from the target, we get a 94.44% accuracy
When predicting the target from the database, we get a 92.59% accuracy
When predicting the database from the target, we get a 94.44% accuracy
When predicting the target from the database, we get a 90.74% accuracy
When predicting the database from the target, we get a 88.88% accuracy
When predicting the target from the database, we get a 90.74% accuracy
When predicting the database from the target, we get a 88.88% accuracy
When predicting the target from the database, we get a 94.44% accuracy
When predicting the database from the target, we get a 88.88% accuracy
When predicting the target from the database, we get a 85.18% accuracy
When predicting the database from the target, we get a 85.18% accuracy
When p

When predicting the target from the database, we get a 83.33% accuracy
When predicting the database from the target, we get a 79.62% accuracy
When predicting the target from the database, we get a 62.96% accuracy
When predicting the database from the target, we get a 62.96% accuracy
When predicting the target from the database, we get a 64.81% accuracy
When predicting the database from the target, we get a 68.51% accuracy
When predicting the target from the database, we get a 66.66% accuracy
When predicting the database from the target, we get a 62.96% accuracy
When predicting the target from the database, we get a 62.96% accuracy
When predicting the database from the target, we get a 66.66% accuracy
When predicting the target from the database, we get a 90.74% accuracy
When predicting the database from the target, we get a 85.18% accuracy
When predicting the target from the database, we get a 68.51% accuracy
When predicting the database from the target, we get a 66.66% accuracy
When p

In [11]:
dppca = pd.DataFrame(target_from_database)
dppca['dataset1']=dataset1
dppca['dataset2']=dataset2
dppca.to_csv("2023_aperiodic_control_subj_short_recordings_NEW.csv", header=False, index=False)

self_corr_pd = pd.DataFrame(self_corr)
self_corr_pd.to_csv("2023_aperiodic_CTL_short_recordings_NEW_self_corr.csv", header=False, index=False)

cross_corr_pd = pd.DataFrame(cross_corr)
cross_corr_pd.to_csv("2023_aperiodic_CTL_short_recordings_NEW_cross_corr.csv", header=False, index=False)


In [12]:
# Parameters
n_subs = 79 # Change here to get number of participants! 
n_feats = int(68*115)
n_measurements = 2

# Warangle data set into two big feature matrices
def prune_subject_csv(filename):
    '''
    This function takes in the subject's csv file from MATLAB, takes out the 
    doubled correlations (because of symmetry) and outputs a numpy array ready to be concatenated
    in the grand feature matrix
    Args:
        filename (string): Name of the csv matrix
    Returns: 
        sub_feat (np.array): Subject's features 
    '''
    #print(filename)
    #print(filename[73:78])
    sub_feat = np.zeros([1, (n_feats)+1]) # Number of unique values in corr matrix + subject label
    psd_matrix = pd.read_csv(filename, header=None)
    mat=np.asmatrix(psd_matrix)
    sub_feat[0, :-1]=mat[:,0:451].flatten()
    sub_feat[0, -1] = int(filename[71:76])    
    return sub_feat


dataset1= [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3,3,3,3,3,3,3, 4, 4, 4,4,4,4,4,4,4, 5,5,5,5,5,5,5,5, 6,6,6,6,6,6,6,7,7,7,7,7,7,8,8,8,8,8,9,9,9,9,10,10,10, 11,11, 12]
dataset2= [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 4, 5, 6, 7,8, 9, 10, 11, 12, 13,  5, 6, 7,8, 9, 10, 11, 12, 13, 6,7, 8, 9, 10, 11, 12, 13, 7, 8, 9, 10, 11, 12, 13, 8, 9, 10, 11, 12, 13, 9,10 ,11,12,13,10,11,12,13, 11,12,13, 12, 13,13 ]

target_from_database = np.zeros([78, 2])
self_corr = np.zeros([79, 78])
cross_corr = np.zeros([79, 78])
for j in range(0,78):

    # Get n subjects: both training and testing datasets
    onlyfiles = [f for f in listdir('/Users/jason/Documents/PD_fingerprinting/2023_data/short_aperiodic/') if isfile(join('/Users/jason/Documents/PD_fingerprinting/2023_data/short_aperiodic/', f))]
    sub_target = np.zeros([n_subs, (n_feats)+1])
    sub_database = np.zeros([n_subs, (n_feats)+1])
    iv = 0
    it = 0
    for iFile in sorted(onlyfiles)[702:((n_subs +54)*13)]: 
        sub = '/Users/jason/Documents/PD_fingerprinting/2023_data/short_aperiodic/' + iFile
        #print(sub)
        #print(re.sub("[^0-9]", "", sub[97:100]))
        if re.sub("[^0-9]", "", sub[95:98]) == str(dataset1[j]):
            sub_database[it, :] = prune_subject_csv(sub)
            it += 1
        if re.sub("[^0-9]", "", sub[95:98]) == str(dataset2[j]):
            sub_target[iv, :] = prune_subject_csv(sub)
            iv += 1


    # Correlations can be computed as the dot product between two z-scored vectors
    z_target = sp.stats.zscore(sub_target[:, :-1], axis = 1)
    z_database = sp.stats.zscore(sub_database[:,:-1], axis = 1)
    predictions = z_target.dot(z_database.transpose()) / (sub_database.shape[1] - 1) # target, database
    target_from_database[j,0] = accuracy_score(range(n_subs), predictions.argmax(axis = 1))
    target_from_database[j,1] = accuracy_score(range(n_subs), predictions.argmax(axis = 0))
    self_corr[0:133,j] = predictions.diagonal()
    np.fill_diagonal(predictions, 0)
    cross_corr[0:133,j] = predictions.mean(axis=1 )*(79/78)

    print('When predicting the target from the database, we get a ' + str(target_from_database[j,0]*100)[0:5] + '% accuracy')
    print('When predicting the database from the target, we get a ' + str(target_from_database[j,1]*100)[0:5] + '% accuracy')


When predicting the target from the database, we get a 98.73% accuracy
When predicting the database from the target, we get a 93.67% accuracy
When predicting the target from the database, we get a 87.34% accuracy
When predicting the database from the target, we get a 83.54% accuracy
When predicting the target from the database, we get a 88.60% accuracy
When predicting the database from the target, we get a 87.34% accuracy
When predicting the target from the database, we get a 83.54% accuracy
When predicting the database from the target, we get a 83.54% accuracy
When predicting the target from the database, we get a 83.54% accuracy
When predicting the database from the target, we get a 78.48% accuracy
When predicting the target from the database, we get a 82.27% accuracy
When predicting the database from the target, we get a 78.48% accuracy
When predicting the target from the database, we get a 82.27% accuracy
When predicting the database from the target, we get a 81.01% accuracy
When p

When predicting the target from the database, we get a 73.41% accuracy
When predicting the database from the target, we get a 75.94% accuracy
When predicting the target from the database, we get a 56.96% accuracy
When predicting the database from the target, we get a 60.75% accuracy
When predicting the target from the database, we get a 51.89% accuracy
When predicting the database from the target, we get a 56.96% accuracy
When predicting the target from the database, we get a 49.36% accuracy
When predicting the database from the target, we get a 53.16% accuracy
When predicting the target from the database, we get a 55.69% accuracy
When predicting the database from the target, we get a 58.22% accuracy
When predicting the target from the database, we get a 83.54% accuracy
When predicting the database from the target, we get a 84.81% accuracy
When predicting the target from the database, we get a 63.29% accuracy
When predicting the database from the target, we get a 67.08% accuracy
When p

In [57]:
dppca = pd.DataFrame(target_from_database)
dppca['dataset1']=dataset1
dppca['dataset2']=dataset2
dppca.to_csv("2023_aperiodic_PKD_subj_short_recordings_NEW.csv", header=False, index=False)

self_corr_pd = pd.DataFrame(self_corr)
self_corr_pd.to_csv("2023_aperiodic_PKD_short_recordings_NEW_self_corr.csv", header=False, index=False)

cross_corr_pd = pd.DataFrame(cross_corr)
cross_corr_pd.to_csv("2023_aperiodic_PKD_short_recordings_NEW_cross_corr.csv", header=False, index=False)


In [15]:

# Parameters
n_subs = 133 # Change here to get number of participants! 
n_feats = int(68*115)


n_measurements = 2
n_subs_2=55
self_id_clinical=np.zeros([n_subs,78])
self_corr_clinical=np.zeros([n_subs,78])
acc_over=np.zeros([78,2])
self_acc=np.zeros([n_subs,2])
clinic=np.zeros(n_subs)
cross_corr_clinical=np.zeros([n_subs,78])

# Warangle data set into two big feature matrices
def prune_subject_csv(filename):
    '''
    This function takes in the subject's csv file from MATLAB, takes out the 
    doubled correlations (because of symmetry) and outputs a numpy array ready to be concatenated
    in the grand feature matrix
    Args:
        filename (string): Name of the csv matrix
    Returns: 
        sub_feat (np.array): Subject's features 
    '''
    #print(filename)
    #print(filename[21:26])
    sub_feat = np.zeros([1, (n_feats)+1]) # Number of unique values in corr matrix + subject label
    psd_matrix = pd.read_csv(filename, header=None)
    mat=np.asmatrix(psd_matrix)
    sub_feat[0, :-1]=mat[:,0:115].flatten()
    sub_feat[0, -1] = int(filename[71:76])    
    return sub_feat

dataset1= [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3,3,3,3,3,3,3, 4, 4, 4,4,4,4,4,4,4, 5,5,5,5,5,5,5,5, 6,6,6,6,6,6,6,7,7,7,7,7,7,8,8,8,8,8,9,9,9,9,10,10,10, 11,11, 12]
dataset2= [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 4, 5, 6, 7,8, 9, 10, 11, 12, 13,  5, 6, 7,8, 9, 10, 11, 12, 13, 6,7, 8, 9, 10, 11, 12, 13, 7, 8, 9, 10, 11, 12, 13, 8, 9, 10, 11, 12, 13, 9,10 ,11,12,13,10,11,12,13, 11,12,13, 12, 13,13 ]

for j in range(0,78):

    # Get n subjects: both training and testing datasets
    onlyfiles = [f for f in listdir('/Users/jason/Documents/PD_fingerprinting/2023_data/short_aperiodic/') if isfile(join('/Users/jason/Documents/PD_fingerprinting/2023_data/short_aperiodic/', f))]
    sub_target = np.zeros([n_subs, (n_feats)+1])
    sub_database = np.zeros([n_subs, (n_feats)+1])
    iv = 0
    it = 0
    for iFile in sorted(onlyfiles)[0:(n_subs*13)]: 
        sub = '/Users/jason/Documents/PD_fingerprinting/2023_data/short_aperiodic/' + iFile
        #print(sub)
        #print(re.sub("[^0-9]", "", sub[45:47]))
        if re.sub("[^0-9]", "", sub[95:98]) == str(dataset1[j]):
            sub_database[it, :] = prune_subject_csv(sub)
            it += 1
        if re.sub("[^0-9]", "", sub[95:98]) == str(dataset2[j]):
            sub_target[iv, :] = prune_subject_csv(sub)
            iv += 1


    for i in range(54,133):
        np_arr1=np.array(i)
        np_arr2=np.array(range(0,54))
        index=np.append(np_arr2, np_arr1)

        sub_target_1=sub_target[index, :]
        sub_database_1=sub_database[index, :]

        # Correlations can be computed as the dot product between two z-scored vectors
        z_target = sp.stats.zscore(sub_target_1[:, :-1], axis = 1)
        z_database = sp.stats.zscore(sub_database_1[:,:-1], axis = 1)
        predictions = z_target.dot(z_database.transpose()) / (sub_database.shape[1] - 1) # target, database
        target_from_database = accuracy_score(range(n_subs_2), predictions.argmax(axis = 1))
        database_from_target = accuracy_score(range(n_subs_2), predictions.argmax(axis = 0))

        self_id = np.diagonal(sp.stats.zscore(predictions, axis = 1))
        cor_predictions=np.diagonal(predictions)
        clinic[i]=predictions.mean(axis=1)[-1]
        cross_corr_predic = predictions.mean(axis=1 )*(55/54)

        self_id_clinical[i,j]=self_id[-1]
        self_corr_clinical[i,j]=cor_predictions[-1]
        cross_corr_clinical[i,j]=cross_corr_predic[-1]
        self_acc[i,0]= (predictions.argmax(axis = 1)[-1]==54)
        self_acc[i,1]= (predictions.argmax(axis = 0)[-1]==54)
    acc_over[j,0]=  (sum(self_acc)/79)[0]
    acc_over[j,1]=  (sum(self_acc)/79)[1]
    
    

In [16]:

df = pd.DataFrame(cross_corr_clinical)
df.to_csv("2023_aperiodic_PDVSCT_short_recordings_NEW_cross_corr.csv")
df = pd.DataFrame(self_corr_clinical)
df.to_csv("2023_aperiodic_PDVSCT_short_recordings_NEW_self_corr.csv")
df = pd.DataFrame(acc_over)
df.to_csv("2023_aperiodic_PDVSCT_subj_short_recordings_NEW.csv")